# Prompt Engineering

You can guide the model to improve its response for your task through specific insturctions or by including different kinds of information, or "context":
- Providing examples of the task that your trying to carry out (in-context learning)
- Specifying how to format responses
- Requesting that the model assumes a certain role or persona
- Including additional information or data for the model to use in its response

In [1]:
from utils import llama, llama_chat

/Users/sinan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## In-context learning

So far, you have been stating the instruction explicitly in the prompt:

In [2]:
prompt = """
What is the sentiment of:
Hi Amit, thanks for the thoughtful birthday card!
"""
response = llama(prompt)
print(response)

  The sentiment of the message "Hi Amit, thanks for the thoughtful birthday card!" is positive. The use of the word "thoughtful" implies that the sender appreciated the effort put into the card, and the tone is friendly and sincere.


## Zero shot prompting

- You are prompting the model to see if it can infer the task from the structure of your prompt.
- In zero-shot prompting, you only provide the structure to the model, but without any examples of the completed task.

In [3]:
prompt = """
Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?
"""
response = llama(prompt)
print(response)

  The sentiment of the message is "Appreciation" or "Gratitude". The sender is expressing their appreciation for the birthday card that Amit sent.


Note that the model doesn't give a negative or positive sentiment.

## Few-shot prompting

- In few-shot prompting, you not only provide the structure to the model, but also two or more examples.
- You are prompting the model to see if it can infer the task from the structure, as well as the examples in your prompt.

In [5]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?
"""
response = llama(prompt, verbose=True)

Prompt:
[INST]
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?
[/INST]

model: togethercomputer/llama-2-7b-chat


In [6]:
print(response)

  Sure, here are the sentiments for each message:

1. Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative
2. Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive
3. Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: Positive


Model is able to give us the right sentiment. However, it repeats the examples. Ideally, we would want to get a short response, like a single word. Let's add this the the prompt.

## Specifying the output format

We can specify the output format to give us a single word.

In [7]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?

Give a one word response.
"""
response = llama(prompt)
print(response)

  Sure! Here are the one-word responses for each message:

1. Negative: Disappointed
2. Positive: Excited
3. ? (Uncertain): Grateful


As you see, llama is totally confused. Maybe because we have been using the smallest llama model and its capacity may not be enough. Let's try with a very large llama model:

In [8]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?

Give a one word response.
"""
response = llama(prompt, model="togethercomputer/llama-2-70b-chat")
print(response)

  Positive


Look at that! It did. Let's try again with the smaller model. This time let's restrict the possible responses to be negative, positive, and neutral.

In [16]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?

Respond with either 'Positive' or 'Negative' or 'Neutral'
"""
response = llama(prompt)
print(response)

  Sure, I'd be happy to help! Here are my responses:

1. Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative
2. Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive
3. Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: Positive


Still, we can't get a one word. Maybe llama 7B is not good for this task.

## Role prompting

- Roles give context to LLMs what type of answers are desired.
- Llama 2 often gives more consistent responses when provided with a role.
- First, try standard prompt and see the response.

In [17]:
prompt = """
How can I answer this question from my friend:
What is the meaning of life?
"""
response = llama(prompt)
print(response)

  The question of the meaning of life is a complex and philosophical one that has been debated throughout human history. There are many different perspectives and interpretations on what the meaning of life is, and there is no one definitive answer. However, here are some possible ways to approach this question:

1. Religious or spiritual perspective: Many people believe that the meaning of life is to fulfill a divine or spiritual purpose, whether that be to follow a set of moral guidelines, to achieve spiritual enlightenment, or to fulfill a specific mission or calling.
2. Personal fulfillment: Some people believe that the meaning of life is to find personal fulfillment and happiness, whether that be through relationships, career, hobbies, or other activities.
3. Social or cultural perspective: From a social or cultural perspective, the meaning of life may be tied to the values and beliefs of one's community or society. For example, some cultures place a strong emphasis on family and 

Now, try it by giving the model a "role", and within the role, a "tone" using which it should respond with.

In [18]:
role = """
Your role is a life coach \
who gives advice to people about living a good life.\
You attempt to provide unbiased advice.
You respond in the tone of an English pirate.
"""

prompt = f"""
{role}
How can I answer this question from my friend:
What is the meaning of life?
"""
response = llama(prompt)
print(response)

  Shiver me timbers! Yer lookin' fer the meaning o' life, eh? Well, matey, that be a question that's been puzzlin' the greatest minds on the high seas fer centuries! *adjusts eye patch*

Now, I ain't one to give ye a straight answer, but I'll share me thoughts with ye. The meaning o' life, me hearty, be different fer each and every one o' us. It be the sum o' all yer experiences, the memories ye make, the adventures ye have, and the treasure ye find along the way! *winks*

Ye see, life be a great big ocean, and ye be a ship sailin' through it. Ye gotta chart yer own course, follow yer heart, and navigate through the storms and calm seas. The meaning o' life be findin' yer own treasure, me matey! *adjusts hat*

So, don't be lookin' fer a definitive answer, or a treasure map that'll lead ye straight to the meaning o' life. It be a journey, a adventure, a treasure hunt, if ye will! *winks*

Now, go forth and find yer own treasure, me hearty! And remember, the meaning o' life be whatever y

## Summarization

- Summarizing a large text is another common use case for LLMs.

In [19]:
email = """
Dear Amit,

An increasing variety of large language models (LLMs) are open source, or close to it. The proliferation of models with relatively permissive licenses gives developers more options for building applications.

Here are some different ways to build applications based on LLMs, in increasing order of cost/complexity:

Prompting. Giving a pretrained LLM instructions lets you build a prototype in minutes or hours without a training set. Earlier this year, I saw a lot of people start experimenting with prompting, and that momentum continues unabated. Several of our short courses teach best practices for this approach.
One-shot or few-shot prompting. In addition to a prompt, giving the LLM a handful of examples of how to carry out a task — the input and the desired output — sometimes yields better results.
Fine-tuning. An LLM that has been pretrained on a lot of text can be fine-tuned to your task by training it further on a small dataset of your own. The tools for fine-tuning are maturing, making it accessible to more developers.
Pretraining. Pretraining your own LLM from scratch takes a lot of resources, so very few teams do it. In addition to general-purpose models pretrained on diverse topics, this approach has led to specialized models like BloombergGPT, which knows about finance, and Med-PaLM 2, which is focused on medicine.
For most teams, I recommend starting with prompting, since that allows you to get an application working quickly. If you’re unsatisfied with the quality of the output, ease into the more complex techniques gradually. Start one-shot or few-shot prompting with a handful of examples. If that doesn’t work well enough, perhaps use RAG (retrieval augmented generation) to further improve prompts with key information the LLM needs to generate high-quality outputs. If that still doesn’t deliver the performance you want, then try fine-tuning — but this represents a significantly greater level of complexity and may require hundreds or thousands more examples. To gain an in-depth understanding of these options, I highly recommend the course Generative AI with Large Language Models, created by AWS and DeepLearning.AI.

(Fun fact: A member of the DeepLearning.AI team has been trying to fine-tune Llama-2-7B to sound like me. I wonder if my job is at risk? 😜)

Additional complexity arises if you want to move to fine-tuning after prompting a proprietary model, such as GPT-4, that’s not available for fine-tuning. Is fine-tuning a much smaller model likely to yield superior results than prompting a larger, more capable model? The answer often depends on your application. If your goal is to change the style of an LLM’s output, then fine-tuning a smaller model can work well. However, if your application has been prompting GPT-4 to perform complex reasoning — in which GPT-4 surpasses current open models — it can be difficult to fine-tune a smaller model to deliver superior results.

Beyond choosing a development approach, it’s also necessary to choose a specific model. Smaller models require less processing power and work well for many applications, but larger models tend to have more knowledge about the world and better reasoning ability. I’ll talk about how to make this choice in a future letter.

Keep learning!

Andrew
"""

In [20]:
prompt = f"""
Summarize this email and extract some key points.
What did the author say about llama models?:

email: {email}
"""

response = llama(prompt)
print(response)

  The author of the email discusses the use of large language models (LLMs) and provides an overview of different ways to build applications based on these models. The key points extracted from the email are:

1. Increasing number of LLMs are open source or close to it, giving developers more options for building applications.
2. Different ways to build applications based on LLMs, in increasing order of cost/complexity: prompting, one-shot or few-shot prompting, fine-tuning, and pretraining.
3. Prompting allows developers to build a prototype in minutes or hours without a training set.
4. One-shot or few-shot prompting gives the LLM a handful of examples of how to carry out a task, which sometimes yields better results.
5. Fine-tuning an LLM that has been pretrained on a lot of text can be done by training it further on a small dataset of your own.
6. Pretraining a model from scratch takes a lot of resources, so it is not recommended for most teams.
7. Specialized models like Bloomberg

## Providing new information in the prompt

- A model's knowledge of the world ends at the moment of its training - so it won't know about more recent events.
- Llama 2 was released for research and commercial use on July 18, 2023, and its training ended some time before that date.
- Ask the model about an event, in this case, FIFA Women's World Cup 2023, which started on July 20, 2023, and see how the model responses.

In [21]:
prompt = """
Who won the 2023 Women's World Cup?
"""
response = llama(prompt)
print(response)

  The 2023 Women's World Cup has not yet taken place, as it is scheduled to be held in 2023. The tournament is organized by FIFA (Fédération Internationale de Football Association) and is held every four years. The winner of the 2023 Women's World Cup will be determined by the team that wins the final match, which is expected to take place in July 2023.


- As you can see, the model still thinks that the tournament is yet to be played, even though you are now in 2024!
- Another thing to note is, July 18, 2023 was the date the model was released to public, and it was trained even before that, so it only has information upto that point. The response says, "the final match is scheduled to take place in July 2023", but the final match was played on August 20, 2023.
- You can provide the model with information about recent events, in this case text from Wikipedia about the 2023 Women's World Cup.

In [22]:
context = """
The 2023 FIFA Women's World Cup (Māori: Ipu Wahine o te Ao FIFA i 2023)[1] was the ninth edition of the FIFA Women's World Cup, the quadrennial international women's football championship contested by women's national teams and organised by FIFA. The tournament, which took place from 20 July to 20 August 2023, was jointly hosted by Australia and New Zealand.[2][3][4] It was the first FIFA Women's World Cup with more than one host nation, as well as the first World Cup to be held across multiple confederations, as Australia is in the Asian confederation, while New Zealand is in the Oceanian confederation. It was also the first Women's World Cup to be held in the Southern Hemisphere.[5]
This tournament was the first to feature an expanded format of 32 teams from the previous 24, replicating the format used for the men's World Cup from 1998 to 2022.[2] The opening match was won by co-host New Zealand, beating Norway at Eden Park in Auckland on 20 July 2023 and achieving their first Women's World Cup victory.[6]
Spain were crowned champions after defeating reigning European champions England 1–0 in the final. It was the first time a European nation had won the Women's World Cup since 2007 and Spain's first title, although their victory was marred by the Rubiales affair.[7][8][9] Spain became the second nation to win both the women's and men's World Cup since Germany in the 2003 edition.[10] In addition, they became the first nation to concurrently hold the FIFA women's U-17, U-20, and senior World Cups.[11] Sweden would claim their fourth bronze medal at the Women's World Cup while co-host Australia achieved their best placing yet, finishing fourth.[12] Japanese player Hinata Miyazawa won the Golden Boot scoring five goals throughout the tournament. Spanish player Aitana Bonmatí was voted the tournament's best player, winning the Golden Ball, whilst Bonmatí's teammate Salma Paralluelo was awarded the Young Player Award. England goalkeeper Mary Earps won the Golden Glove, awarded to the best-performing goalkeeper of the tournament.
Of the eight teams making their first appearance, Morocco were the only one to advance to the round of 16 (where they lost to France; coincidentally, the result of this fixture was similar to the men's World Cup in Qatar, where France defeated Morocco in the semi-final). The United States were the two-time defending champions,[13] but were eliminated in the round of 16 by Sweden, the first time the team had not made the semi-finals at the tournament, and the first time the defending champions failed to progress to the quarter-finals.[14]
Australia's team, nicknamed the Matildas, performed better than expected, and the event saw many Australians unite to support them.[15][16][17] The Matildas, who beat France to make the semi-finals for the first time, saw record numbers of fans watching their games, their 3–1 loss to England becoming the most watched television broadcast in Australian history, with an average viewership of 7.13 million and a peak viewership of 11.15 million viewers.[18]
It was the most attended edition of the competition ever held.
"""

In [23]:
prompt = f"""
Given the following context, who won the 2023 Women's World cup?
context: {context}
"""
response = llama(prompt)
print(response)

  Based on the information provided in the context, Spain won the 2023 Women's World Cup.


Another example:

In [27]:
prompt = """
Which AI assistant will Microsoft's Copilot use?
"""
response = llama(prompt)
print(response)

  Microsoft has not yet officially announced which AI assistant will be used in its Copilot feature. However, there are several AI assistants that Microsoft has worked with in the past, including:

1. Cortana: Microsoft's own AI assistant, which is integrated into Windows 10 and other Microsoft products.
2. Bing: Microsoft's search engine, which uses AI to provide search results and answer questions.
3. Azure Cognitive Services: A suite of AI services offered by Microsoft, including speech recognition, language translation, and image recognition.
4. Microsoft Bot Framework: A toolkit for building chatbots and other conversational AI experiences.

It's possible that Microsoft may use one of these AI assistants in Copilot, or they may develop a new AI assistant specifically for the feature. We will have to wait and see what Microsoft announces in the future.


In [28]:
context = """
Copilot, Microsoft's AI assistant in Windows 11 and Bing, will begin using GPT-4o, the latest OpenAI technology that's also powering the paid tier of ChatGPT, the company said at its AI Era event in Seattle on Monday. This newest version of Copilot will be able to answer questions and generate text with greater precision and accuracy, while also having voice and computer vision support. For example, it can be used to open apps or edit photos. Copilot will also be visible in the Windows 11 taskbar. 
"""
query = "Which AI assistant will Microsoft's Copilot use?"

prompt = f"""
Given the following context,
{query}

context: {context}
"""
response = llama(prompt,
                 verbose=True)
print(response)

Prompt:
[INST]
Given the following context,
Which AI assistant will Microsoft's Copilot use?

context: 
Copilot, Microsoft's AI assistant in Windows 11 and Bing, will begin using GPT-4o, the latest OpenAI technology that's also powering the paid tier of ChatGPT, the company said at its AI Era event in Seattle on Monday. This newest version of Copilot will be able to answer questions and generate text with greater precision and accuracy, while also having voice and computer vision support. For example, it can be used to open apps or edit photos. Copilot will also be visible in the Windows 11 taskbar. 

[/INST]

model: togethercomputer/llama-2-7b-chat
  Based on the context provided, Microsoft's Copilot will use GPT-4, the latest OpenAI technology.


## Chain of thought prompting

LLMs can perform better at reasoning and logic problems if you ask them to break the problem down into smaller steps. This is known as chain-of-thought prompting.

In [29]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?
"""
response = llama(prompt)
print(response)

  Yes, all 15 people can get to the restaurant by car or motorcycle.

Here's how:

* Two people with cars can fit 5 people each, so they can take 10 people in total.
* Two people with motorcycles can fit 2 people each, so they can take 4 people in total.

That means there are 10 people who can go by car and 4 people who can go by motorcycle, for a total of 14 people who can get to the restaurant.

The remaining 1 person can either walk or find another mode of transportation.


Modify the prompt to ask the model to "think step by step" about the math problem you provided.

In [30]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?

Think step by step.
"""
response = llama(prompt)
print(response)

  Sure, let's break it down step by step to see if we can get everyone to the restaurant:

1. Number of people who want to go to the restaurant: 15
2. Number of people who have cars: 2
3. Number of people who can fit in each car: 5
4. Number of motorcycles available: 2
5. Number of people who can fit in each motorcycle: 2

Now, let's see if we can accommodate everyone:

1. Car 1: 2 people who want to go to the restaurant (driver and passenger)
2. Car 2: 5 people who want to go to the restaurant (driver and 4 passengers)
3. Motorcycle 1: 2 people who want to go to the restaurant (driver and passenger)
4. Motorcycle 2: 2 people who want to go to the restaurant (driver and passenger)

Total number of people who can get to the restaurant: 12 (6 from Car 1, 5 from Car 2, and 1 from Motorcycle 1)

Unfortunately, we have 3 more people who want to go to the restaurant than we have available transportation. So, no, we cannot all get to the restaurant by car or motorcycle.


Provide model with additional instructions

In [31]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?

Think step by step.
Explain each intermediate step.
Only when you are done with all your steps,
provide the answer based on your intermediate steps.
"""
response = llama(prompt)
print(response)

  Of course! Let's break down the problem step by step to determine if all 15 people can get to the restaurant by car or motorcycle:

Step 1: Identify the number of people who can be accommodated by each car:

* Two cars are available, and each car can seat 5 people.
* Therefore, a total of 10 people can be accommodated by the two cars.

Step 2: Identify the number of people who can be accommodated by each motorcycle:

* Two motorcycles are available, and each motorcycle can fit 2 people.
* Therefore, a total of 4 people can be accommodated by the two motorcycles.

Step 3: Determine the total number of people who can be accommodated for the trip:

* Add the number of people who can be accommodated by the cars (10) and the number of people who can be accommodated by the motorcycles (4):

10 + 4 = 14

Step 4: Check if the total number of people who can be accommodated is equal to the number of people who want to go to the restaurant:

* 14 is less than 15 (the number of people who want t

The order of instructions matters! Ask the model to "answer first" and "explain later" to see how the output changes.

In [32]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?
Think step by step.
Provide the answer as a single yes/no answer first.
Then explain each intermediate step.
"""

response = llama(prompt)
print(response)

  Yes, all 15 people can get to the restaurant by car or motorcycle. Here's how:

Step 1: Two people with cars can fit 10 people (5 people per car x 2 cars).
Step 2: Two people with motorcycles can fit 4 people (2 people per motorcycle x 2 motorcycles).
Step 3: Combine the people who can fit in cars and those who can fit in motorcycles: 10 + 4 = 14 people can get to the restaurant.

Therefore, the answer is yes, all 15 people can get to the restaurant by car or motorcycle.


Since LLMs predict their answer one token at a time, the best practice is to ask them to think step by step, and then only provide the answer after they have explained their reasoning.